In [1]:
import pandas as pd
import numpy as np
import sys
import os
import re
import datetime 
import networkx
import json
from konlpy.tag import Komoran, Kkma, Twitter
from kss import split_sentences
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def tokenize(content):
    #split sentence
    sent_list = [sent for sent in split_sentences(''.join(content)) if sent is not None]
    return sent_list

In [3]:
class TextRank:
    def __init__(self, window = 5, coef = 1.0, threshold=0.005):
        self.posTagger = Twitter()
        self.graph2 = []
        self.nouns = []
        self.sentences = []
 
    def loadSentence(self, sentenceIter):    
        self.sentences = tokenize(sentenceIter)   
        for sentence in self.sentences:
            if sentence is not '':
                self.nouns.append(' '.join([noun for noun in self.posTagger.nouns(str(sentence)) 
                                       if len(noun) > 1]))
    
    def get_rank(self, d=0.85):
        matrix_size = self.graph2.shape[0]
        for id in range(matrix_size):
            self.graph2[id, id] = 0
            link_sum = np.sum(self.graph2[:,id])
            if link_sum != 0:
                self.graph2[:, id] /= link_sum
            self.graph2[:, id] *= -d
            self.graph2[id, id] = 1
        graph_mat = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(self.graph2, graph_mat)
        return {idx: r[0] for idx, r in enumerate(ranks)}
    
    def build(self, sentence):
        tfidf = TfidfVectorizer()
        count = CountVectorizer()
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        self.graph2 = np.dot(tfidf_mat, tfidf_mat.T)
 

    def summarize(self):
        summary = []
        index=[]

        idx_rank = self.get_rank()
        sorted_rank_idx = sorted(idx_rank, key=lambda k: idx_rank[k], reverse=True)
        for idx in sorted_rank_idx[:3]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return ' '.join(summary)

<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/k5/_4q9447d6f727nxck_52wgy40000gn/T/ipykernel_42206/2225168286.py:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [4]:
def summarise_contents(x):
    tr = TextRank(5, 1.0, 0.005) #window, coef, threshold

    tr.loadSentence(tokenize(x))

    tr.build(x)
    ranks = tr.get_rank()
#     ranks = tr.rank()
    return tr.summarize()

In [5]:
with open("../data.json", 'r') as f:
    data = json.load(f)

In [6]:
content = data[3]["content"]

In [7]:
summarise_contents(data[3]["content"])

/Users/jeongseog-yeong/opt/anaconda3/lib/python3.9/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
[Korean Sentence Splitter]: Initializing Pynori...


'주요 변경 내용은 가산금리 인하(은행별 상이)와 상환방식 변경, 금융기관(국민은행) 변경 등이다. 특례보증 지원제도는 담보력이 부족해 제도권 은행대출이 사실상 어려운 지역의 소상공인에게 정읍시와 전북신용보증재단이 대신 보증서를 발급해줌으로써 운영자금 대출이 가능하도록 하는 제도이다. 한편, 특례보증 지원 신청은 자금소진 시까지 운영된다.'